<a id="im"></a>
# <center>IMPORTING LIBRARIES</center> 

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from sklearn.metrics import confusion_matrix , classification_report 
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import roc_curve, auc, roc_auc_score

from IPython.display import clear_output
import warnings
warnings.filterwarnings('ignore')

<a id="hp"></a>
# <center>HYPERPARAMETRERS AND DIRECTORIES</center>

In [45]:
train_dir = "train"
test_dir = "test"

SEED = 12
IMG_HEIGHT = 48
IMG_WIDTH = 48
BATCH_SIZE = 64
EPOCHS = 15
FINE_TUNING_EPOCHS = 10
LR = 0.01
NUM_CLASSES = 6
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Anger', 'Fear', 'Happy', 'Neutral', 'Sadness', "Surprise"]

<a id="data"></a>
# <center> DATA LOADING AND PRE-PROCESSING</center>

In [46]:
def to_grayscale_then_rgb(image):
    image = tf.image.rgb_to_grayscale(image)
    image = tf.image.grayscale_to_rgb(image)
    return image

# preprocess_fun = tf.keras.applications.densenet.preprocess_input
preprocess_fun = to_grayscale_then_rgb

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rescale = 1./255,
                                   validation_split = 0.2,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0.2,
                                  preprocessing_function=preprocess_fun)

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

validation_generator = test_datagen.flow_from_directory(directory = train_dir,
                                                         target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                         batch_size = BATCH_SIZE,
                                                         shuffle  = True , 
                                                         color_mode = "rgb",
                                                         class_mode = "categorical",
                                                         subset = "validation",
                                                         seed = 12
                                                        )

test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 22631 images belonging to 6 classes.
Found 5654 images belonging to 6 classes.
Found 7067 images belonging to 6 classes.


## Data distribution (count) among differnt emotions

In [47]:
classLabelCounts = [list(train_generator.classes).count(i) for i in np.unique(train_generator.classes)]
for classLabel, classLabelCount in zip(CLASS_LABELS, classLabelCounts):
  print(classLabel, classLabelCount)

Anger 3198
Fear 3280
Happy 5774
Neutral 3974
Sadness 3866
Surprise 2539


<a id="model"></a>
# <center> DenseNet169 Transfer Learning  </center>

In [48]:
def feature_extractor(inputs):
    feature_extractor = tf.keras.applications.DenseNet169(input_shape=(IMG_HEIGHT,IMG_WIDTH, 3),
                                               include_top=False,
                                               weights="imagenet")(inputs)
    
    return feature_extractor

def classifier(inputs):
    x = tf.keras.layers.GlobalAveragePooling2D()(inputs)
    x = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.3)(x)
    x = tf.keras.layers.Dense(1024, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation="relu", kernel_regularizer = tf.keras.regularizers.l2(0.01))(x)
    x = tf.keras.layers.Dropout(0.5) (x)
    x = tf.keras.layers.Dense(NUM_CLASSES, activation="softmax", name="classification")(x)
    
    return x

def final_model(inputs):
    densenet_feature_extractor = feature_extractor(inputs)
    classification_output = classifier(densenet_feature_extractor)
    
    return classification_output

def define_compile_model():
    
    inputs = tf.keras.layers.Input(shape=(IMG_HEIGHT ,IMG_WIDTH,3))
    classification_output = final_model(inputs) 
    model = tf.keras.Model(inputs=inputs, outputs = classification_output)
     
    model.compile(optimizer=tf.keras.optimizers.SGD(0.05), 
                loss='categorical_crossentropy',
                metrics = ['accuracy'])
  
    return model

## Summary of model

In [49]:
model = define_compile_model()
clear_output()

# Feezing the feature extraction layers
model.layers[1].trainable = False

model.summary()

Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 densenet169 (Functional)    (None, 1, 1, 1664)        12642880  
                                                                 
 global_average_pooling2d_3   (None, 1664)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dense_9 (Dense)             (None, 512)               852480    
                                                                 
 dropout_9 (Dropout)         (None, 512)               0         
                                                                 
 dense_10 (Dense)            (None, 1024)              525312    
                                                           

<a id="train"></a>
# <center> Training and Fine-Tuning </center> 

## Training model with freezed layers of DenseNer169

In [50]:
earlyStoppingCallback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                         patience=EARLY_STOPPING_CRITERIA,
                                                         verbose= 1 ,
                                                         restore_best_weights=True
                                                        )

history = model.fit(x = train_generator,
                    epochs = EPOCHS ,
                    validation_data = validation_generator , 
                    callbacks= [earlyStoppingCallback])

history = pd.DataFrame(history.history)

Epoch 1/15
354/354 [==============================] - 192s 409ms/step - loss: 17.0379 - accuracy: 0.3178 - val_loss: 12.0032 - val_accuracy: 0.4307
Epoch 2/15
354/354 [==============================] - 136s 385ms/step - loss: 8.9306 - accuracy: 0.4755 - val_loss: 6.5049 - val_accuracy: 0.5004
Epoch 3/15
354/354 [==============================] - 136s 384ms/step - loss: 4.9729 - accuracy: 0.5327 - val_loss: 3.7680 - val_accuracy: 0.5426
Epoch 4/15
354/354 [==============================] - 136s 383ms/step - loss: 3.0195 - accuracy: 0.5644 - val_loss: 2.4380 - val_accuracy: 0.5506
Epoch 5/15
354/354 [==============================] - 136s 384ms/step - loss: 2.0348 - accuracy: 0.5895 - val_loss: 1.7354 - val_accuracy: 0.5858
Epoch 6/15
354/354 [==============================] - 136s 384ms/step - loss: 1.5325 - accuracy: 0.6124 - val_loss: 1.3958 - val_accuracy: 0.6045
Epoch 7/15
354/354 [==============================] - 135s 380ms/step - loss: 1.2770 - accuracy: 0.6237 - val_loss: 1.2827

## Fine Tuning

In [51]:
# Un-Freezing the feature extraction layers for fine tuning 
model.layers[1].trainable = True

model.compile(optimizer=tf.keras.optimizers.SGD(0.001), #lower learning rate
                loss='categorical_crossentropy',
                metrics = ['accuracy'])

history_ = model.fit(x = train_generator,epochs = FINE_TUNING_EPOCHS ,validation_data = validation_generator)
history = history.append(pd.DataFrame(history_.history) , ignore_index=True)

Epoch 1/10
354/354 [==============================] - 184s 389ms/step - loss: 0.7784 - accuracy: 0.7415 - val_loss: 1.0112 - val_accuracy: 0.6558
Epoch 2/10
354/354 [==============================] - 135s 381ms/step - loss: 0.7304 - accuracy: 0.7596 - val_loss: 1.0096 - val_accuracy: 0.6622
Epoch 3/10
354/354 [==============================] - 134s 379ms/step - loss: 0.7063 - accuracy: 0.7682 - val_loss: 1.0136 - val_accuracy: 0.6640
Epoch 4/10
354/354 [==============================] - 135s 381ms/step - loss: 0.6961 - accuracy: 0.7721 - val_loss: 1.0180 - val_accuracy: 0.6654
Epoch 5/10
354/354 [==============================] - 134s 379ms/step - loss: 0.6878 - accuracy: 0.7758 - val_loss: 1.0179 - val_accuracy: 0.6684
Epoch 6/10
354/354 [==============================] - 136s 384ms/step - loss: 0.6769 - accuracy: 0.7782 - val_loss: 1.0213 - val_accuracy: 0.6677
Epoch 7/10
354/354 [==============================] - 135s 381ms/step - loss: 0.6704 - accuracy: 0.7817 - val_loss: 1.0243 -

## Model Evaluation

In [1]:
model.evaluate(test_generator)
preds = model.predict(test_generator)
y_preds = np.argmax(preds , axis = 1 )
y_test = np.array(test_generator.labels)

NameError: ignored

## Confusion Matrix

In [ ]:
cm_data = confusion_matrix(y_test , y_preds)
cm = pd.DataFrame(cm_data, columns=CLASS_LABELS, index = CLASS_LABELS)
cm.index.name = 'Actual'
cm.columns.name = 'Predicted'
plt.figure(figsize = (20,10))
plt.title('Confusion Matrix', fontsize = 20)
sns.set(font_scale=1.2)
ax = sns.heatmap(cm, cbar=False, cmap="Blues", annot=True, annot_kws={"size": 16}, fmt='g')

In [ ]:
model.save('model6.h5')

In [ ]:
! pip install -q kaggle
from google.colab import files

files.upload()
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
! kaggle datasets download -d ananthu017/emotion-detection-fer

In [ ]:
import zipfile
import os

data_r = zipfile.ZipFile('/content/emotion-detection-fer.zip', 'r')
data_r.extractall()

In [ ]:
import shutil, sys 
shutil.rmtree('/content/test/disgusted')